In [7]:
import pandas as pd
import fileinput

In [8]:
# read in data ----- there is also mouse and rat data. i will only be using human for now
df = pd.read_csv('input/phosphoELM_vertebrate_2015-04.dump', sep = '\t')
df = df.dropna(subset = ['kinases'])
df = df[df.species == 'Homo sapiens'] # only using human data
df.head()

,acc,sequence,position,code,pmids,kinases,source,species,entry_date
14,O14543,MVTHSKFPAAGMSRPLDTSLRLKTFSSKSEYQLVVNAVRKLQESGF...,204,Y,12783885,Lck,LTP,Homo sapiens,2006-10-17 12:06:48.16767+02
16,O14543,MVTHSKFPAAGMSRPLDTSLRLKTFSSKSEYQLVVNAVRKLQESGF...,221,Y,15173187,Lck,LTP,Homo sapiens,2006-10-17 12:06:48.271076+02
18,O14746,MPRAPRCRAVRSLLRSHYREVLPLATFVRRLGPQGWRLVQRGDPAA...,824,S,10224060,PKB_group,LTP,Homo sapiens,2004-12-31 00:00:00+01
20,O14746,MPRAPRCRAVRSLLRSHYREVLPLATFVRRLGPQGWRLVQRGDPAA...,227,S,10224060,PKB_group,LTP,Homo sapiens,2004-12-31 00:00:00+01
21,O14746,MPRAPRCRAVRSLLRSHYREVLPLATFVRRLGPQGWRLVQRGDPAA...,707,Y,12808100,SRC,LTP,Homo sapiens,2006-10-17 11:57:52.221615+02


In [9]:
# file with gene symbol and uniprotid
df1 = pd.read_csv('input/genesymboltouniprot_hgnc.txt', sep = '\t')
df1 = df1.dropna()
df1.head()

# map uniprot id to list of approved symbols
di = {} # {uniprot id, [approved symbol(s)]}
for p1, p2 in zip(list(df1.loc[:, 'Approved symbol']), list(df1.loc[:, 'UniProt ID(supplied by UniProt)'])):
    if p2 in di:
        di[p2].append(p1)
    else:
        di[p2] = [p1]

In [10]:
# read in list of human kinases
inp = fileinput.FileInput('input/kinases_human.txt')
pkin = []
for line in inp:
    pkin.append(line.split('\n')[0])


In [11]:
# find ksis -- filtering using pkin because some 'kinases' in table are kinase groups
d = {}
for p1, k in zip(df.loc[:, 'acc'], df.loc[:, 'kinases']):
    if p1 in di:
        p1 = di[p1]
        if k in pkin:
            for x in p1:
                if k not in d:
                    d[k] = {x}
                else:
                    d[k].add(x)
print(len(d))

77


In [12]:
# write to file, do calculations
allnp = open('output_phosphoelm_all_human.gmt', 'w+')
fnp = open('output_phosphoelm_fourplusinterations_human.gmt', 'w+')
uSubs = set()
numKSI = 0
for k in d:
    temp = '{0}_phosphoelm_human'.format(k)
    for i in d[k]:
        numKSI += 1
        uSubs.add(i)
        temp += '\t{0}'.format(i)
    allnp.write('{0}\n'.format(temp))
    if len(d[k]) >= 4:
        fnp.write('{0}\n'.format(temp))
        
print('{0}\n#kin: {1}\t#ksi: {2}\t#usubs: {3}'.format('human', len(d), numKSI, len(uSubs)))

human
#kin: 77	#ksi: 493	#usubs: 359
